# Preview Quiz prompt

In [1]:
from pydantic import BaseModel
from openai import OpenAI
from openai.lib._parsing import type_to_response_format_param
import json
from getpass import getpass

openai_api_key = getpass("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

In [5]:
from jinja2 import Template

prompt_template = Template("""
[PURPOSE]
Translate the "KEY_EXPRESSION" and "SUB_EXPRESSION"

[Example]
```
Input: 
    {
    "DESCRIPTION": "'What's this/that?'를 활용해 사물의 이름을 묻고 답하기.",
    "KEY_EXPRESSION": "A: What's that? B: It's a marker.",
    "SUB_EXPRESSION": "A: What is this? B: It's a ~. A: What is that? B: It's a ~."
  }

Output:
    {
    "key_expression_kor": "A: 저게 뭐야? B: 그건 마커야.",
    "sub_expression_kor": "A: 이것은 뭐야? B: 이것은 ~야. A: 저것은 뭐야? B: 저것은 ~야."
  }
}
```
```
Input: 
    {
    "DESCRIPTION": "I like/don't like ~를 통해 개인의 선호도를 표현하기.",
    "KEY_EXPRESSION": "A: I like cake.",
    "SUB_EXPRESSION": "A: I enjoy ~. A: I'm fond of ~."
  }

Output:
    {
    "key_expression_kor": "A: 나는 케이크를 좋아해.",
    "sub_expression_kor": "A: 나는 ~을 즐겨. A: 나는 ~을 좋아해."
  }
}
```
```
Input: 
    {
    "DESCRIPTION": "고마움에 대한 답변으로 사용되는 표현 익히기.",
    "KEY_EXPRESSION": "No problem!",
    "SUB_EXPRESSION": "It’s no big deal. Don’t mention it."
  }

Output:
    {
    "key_expression_kor": "문제없어!",
    "sub_expression_kor": "별거 아니야. 천만에."
  }
}
```                        


[Input]
```
DESCRIPTION: {{description}}
KEY EXPRESSION: {{key_expression}}
SUB EXPRESSION: {{sub_expression}}
```

""")

In [6]:
prompt = prompt_template.render(
    description = "Is it ~?를 활용해 간단한 확인 및 답하기.",
    key_expression = "A: Is it a bird? B: Yes, it is. ",
    sub_expression = [
        "A: Is it ~? B: Yes, it is. A: Could that be ~? B: Yes, it is. "
    ]
)

In [7]:
print(prompt)


[PURPOSE]
Translate the "KEY_EXPRESSION" and "SUB_EXPRESSION"

[Example]
```
Input: 
    {
    "DESCRIPTION": "'What's this/that?'를 활용해 사물의 이름을 묻고 답하기.",
    "KEY_EXPRESSION": "A: What's that? B: It's a marker.",
    "SUB_EXPRESSION": "A: What is this? B: It's a ~. A: What is that? B: It's a ~."
  }

Output:
    {
    "key_expression_kor": "A: 저게 뭐야? B: 그건 마커야.",
    "sub_expression_kor": "A: 이것은 뭐야? B: 이것은 ~야. A: 저것은 뭐야? B: 저것은 ~야."
  }
}
```
```
Input: 
    {
    "DESCRIPTION": "I like/don't like ~를 통해 개인의 선호도를 표현하기.",
    "KEY_EXPRESSION": "A: I like cake.",
    "SUB_EXPRESSION": "A: I enjoy ~. A: I'm fond of ~."
  }

Output:
    {
    "key_expression_kor": "A: 나는 케이크를 좋아해.",
    "sub_expression_kor": "A: 나는 ~을 즐겨. A: 나는 ~을 좋아해."
  }
}
```
```
Input: 
    {
    "DESCRIPTION": "고마움에 대한 답변으로 사용되는 표현 익히기.",
    "KEY_EXPRESSION": "No problem!",
    "SUB_EXPRESSION": "It’s no big deal. Don’t mention it."
  }

Output:
    {
    "key_expression_kor": "문제없어!",
    "sub_expression_kor": "별거

In [8]:
class Translate(BaseModel):
    key_expression_kor : str
    sub_expression_kor : str

In [9]:
response_format = type_to_response_format_param(Translate)

In [10]:
response_format

{'type': 'json_schema',
 'json_schema': {'schema': {'properties': {'key_expression_kor': {'title': 'Key Expression Kor',
     'type': 'string'},
    'sub_expression_kor': {'title': 'Sub Expression Kor', 'type': 'string'}},
   'required': ['key_expression_kor', 'sub_expression_kor'],
   'title': 'Translate',
   'type': 'object',
   'additionalProperties': False},
  'name': 'Translate',
  'strict': True}}

In [11]:
def completion(prompt : str) -> str:
    response = client.beta.chat.completions.parse(
        model = 'o3-mini',
        reasoning_effort='low',
        messages = [
            {"role" : "system", "content" : "You will provide a Korean translation."},
            {"role" : "user", "content" : prompt}
        ],
        response_format = Translate,
    )
    return response.choices[0].message.parsed

In [12]:
response = completion(prompt)

In [13]:
response

Translate(key_expression_kor='A: 새야? B: 응, 그래.', sub_expression_kor='A: ~야? B: 응, 그래. A: 그게 ~일까? B: 응, 그래.')

In [14]:
response_output = json.dumps(response.dict(), ensure_ascii=False, indent = 4)
print(response_output)

{
    "key_expression_kor": "A: 새야? B: 응, 그래.",
    "sub_expression_kor": "A: ~야? B: 응, 그래. A: 그게 ~일까? B: 응, 그래."
}


In [15]:
print(response)

key_expression_kor='A: 새야? B: 응, 그래.' sub_expression_kor='A: ~야? B: 응, 그래. A: 그게 ~일까? B: 응, 그래.'


# for문 돌려서 Batch API jsonl 파일 형식 만들기

확인 사항
- A1, A2, B1, B2, C1, C2 CEFR level 별로 퀴즈만 만들어지면 되는건가?
- 여섯개의 레벨 모두 만들어야하는지, 일부 레벨(A1, B1, C1)만 만들면 되는지도 결정

1. 커리큘럼 정보 csv 파일 읽어오기
2. for문으로 한줄씩 순회하면서 프롬프트 만들어주기들어주기
3. 렌더링한 프롬프트 파일에 써주기 (처음엔 조금만, ex. 5개 커리큘럼)
4. openai batch API 요청해서 task_id 받아오기 
5. batch task 완료되면 결과 받아오고 검수하기

In [16]:
def QuizMake(data, output_filename):
    jsonl_data = []

    for i in range(len(data)):
        prompt = prompt_template.render(
            description = data.loc[i,"DESCRIPTION"],
            key_expression = data.loc[i,"KEY_EXPRESSION"],
            sub_expression = data.loc[i,"SUB_EXPRESSION"]
        )

        quiz_request = {
            "custom_id" : f"request-{i+1}",
            "method" : "POST",
            "url" : "/v1/chat/completions",
            "body" : {
                "model" : "o3-mini-2025-01-31",
                "messages" : [
                    {"role": "system", "content": "You will provide quizzes related to the topics students have learned, appropriately matching their CEFR level for middle school students."},
                    {"role": "user", "content": prompt}
                ],
                "response_format" : response_format
            }
        }

        jsonl_data.append(quiz_request)

        with open(output_filename, 'w', encoding='utf-8') as jsonl_file:
            for item in jsonl_data:
                jsonl_file.write(json.dumps(item, ensure_ascii=False) + '\n')

    print(f'JSONL 파일 생성 완료 : {output_filename}-{i+1}')

## 초등학교 jsonl 파일

In [18]:
import pandas as pd

raw_data = pd.read_csv("../../../DB/Extract_Elem.csv")
data_Elem3 = raw_data.loc[raw_data['학년'] == '초3']
data_Elem4 = raw_data.loc[raw_data['학년'] == '초4']
data_Elem5 = raw_data.loc[raw_data['학년'] == '초5']
data_Elem6 = raw_data.loc[raw_data['학년'] == '초6']

# index 초기화화
data_Elem3 = data_Elem3.reset_index(drop=True)
data_Elem4 = data_Elem4.reset_index(drop=True)
data_Elem5 = data_Elem5.reset_index(drop=True)
data_Elem6 = data_Elem6.reset_index(drop=True)

display(data_Elem3.head(3))
display(data_Elem4.head(3))
display(data_Elem5.head(3))
display(data_Elem6.head(3))

,Unnamed: 0,textbook ID,학년,출판사,교육과정,단원 번호,단원 명,DESCRIPTION,KEY_EXPRESSION,KEY_EXPRESSION_KOR,SUB_EXPRESSION
0,0,310,초3,YBM(김),2022,1,Hello! I’m Dito,Hello/Hi! 등의 표현을 사용해 만날 때 인사하기.,Hi there!,NaN,Hello! Hey!
1,1,310,초3,YBM(김),2022,2,"Sit Down, Please",극존칭 표현을 통해 요청하고 그에 긍정적으로 응답하기.,"A: Sit down, please. B: Sure.",NaN,A: Please take a seat. B: Certainly. A: Kindly...
2,2,310,초3,YBM(김),2022,3,What’s This?,의문문을 활용해 무슨 물건인지 묻고 답하기.,A: What's this? B: It's a box.,NaN,A: What's that? B: It's a (). A: What is this ...


,Unnamed: 0,textbook ID,학년,출판사,교육과정,단원 번호,단원 명,DESCRIPTION,KEY_EXPRESSION,KEY_EXPRESSION_KOR,SUB_EXPRESSION
0,46,410,초4,YBM(김),2022,1,How Are You?,상대방의 안부를 물어보고 응답하기.,A: How are you doing? B: I'm doing fine.,NaN,A: How have you been? B: I'm feeling (). A: Is...
1,47,410,초4,YBM(김),2022,2,Let’s Play Baseball,Let's를 활용해 제안하고 이에 대해 답하기.,Let's go to the park.,NaN,Let's () together. How about we ()?
2,48,410,초4,YBM(김),2022,3,Where Is My Bag?,위치 묻기 표현을 활용하여 물건의 위치 묻고 답하기.,A: Where is my backpack? B: It's on the table.,NaN,A: Where is my pencil case? B: It's in the dra...


,Unnamed: 0,textbook ID,학년,출판사,교육과정,단원 번호,단원 명,DESCRIPTION,KEY_EXPRESSION,KEY_EXPRESSION_KOR,SUB_EXPRESSION
0,92,503,초5,YBM(김),2015,1,How’s It Going?,How’s it going?을 활용해 안부 묻고 답하기.,How are you doing? — I'm doing well.,NaN,How have you been? — I've been doing well. How...
1,93,503,초5,YBM(김),2015,2,I’m in the Kitchen,의문사 Where를 활용해 사람의 위치 묻고 답하기.,A: Where are you? B: I'm in the living room.,NaN,A: Where is he? B: He's in the () room. A: Whe...
2,94,503,초5,YBM(김),2015,3,Whose Balloon Is This?,Whose의 소유격 의문사를 활용해 물건의 주인이 누구인지 묻고 답하기.,A: Whose bag is this? B: It's Donna's.,NaN,A: To whom does this belong? B: It's ()'s. A: ...


,Unnamed: 0,textbook ID,학년,출판사,교육과정,단원 번호,단원 명,DESCRIPTION,KEY_EXPRESSION,KEY_EXPRESSION_KOR,SUB_EXPRESSION
0,119,603,초6,YBM(김),2015,1,What Grade Are You In?,What grade are you in? — I’m in the sixth grad...,A: What grade are you in? B: I'm in third grade.,NaN,A: What year are you in? B: I am in () grade. ...
1,120,603,초6,YBM(김),2015,2,What Would You Like?,'Would you like ()?'을 활용해 음식에 대해 묻고 답하기.,A: What would you like? B: I'd like strawberry...,NaN,A: What do you want to eat? B: I'd like (). A:...
2,121,603,초6,YBM(김),2015,3,My Favorite Subject Is Science,선호하는 과목에 대해 묻고 답하기.,A: What’s your favorite school subject? B: My ...,NaN,A: What class do you enjoy the most? B: My fav...


In [20]:
QuizMake(data_Elem3, output_filename = 'Translate_Elem3_batch.jsonl')
QuizMake(data_Elem4, output_filename = 'Translate_Elem4_batch.jsonl')
QuizMake(data_Elem5, output_filename = 'Translate_Elem5_batch.jsonl')
QuizMake(data_Elem6, output_filename = 'Translate_Elem6_batch.jsonl')

JSONL 파일 생성 완료 : Translate_Elem3_batch.jsonl-240
JSONL 파일 생성 완료 : Translate_Elem4_batch.jsonl-231
JSONL 파일 생성 완료 : Translate_Elem5_batch.jsonl-99
JSONL 파일 생성 완료 : Translate_Elem6_batch.jsonl-100


## 고등학교 jsonl 파일

In [ ]:
import pandas as pd

raw_data = pd.read_csv("../../DB/Extract_High.csv")
raw_data.head()
raw_data['제목'].value_counts()
# raw_data['출판사'].value_counts()
# data_Mid1 = raw_data.loc[raw_data['학년'] == '고1']
# data_Mid2 = raw_data.loc[raw_data['학년'] == '2']
# data_Mid3 = raw_data.loc[raw_data['학년'] == '중3']

# # index 초기화화
# data_Mid1 = data_Mid1.reset_index(drop=True)
# data_Mid2 = data_Mid2.reset_index(drop=True)
# data_Mid3 = data_Mid3.reset_index(drop=True)

# display(data_Mid1.head(3))
# display(data_Mid2.head(3))
# display(data_Mid3.head(3))

제목
영어        132
영어Ⅰ        48
영어Ⅱ        36
영어 I       24
영어 II      24
공통영어 1     12
공통영어 2     12
영어1        10
영어 2       10
기본영어       10
공통영어 I      8
Name: count, dtype: int64